<p style="font-size:30px; text-align:center; line-height:120%">
    <br> 
        <b>
        COMS 4995 Applied ML
            Homework 4 
        <br></br>
            Wine Reviews: Task 2
        <br></br>
        </b> 
    <br> 
</p>
<p style="font-size:18px; text-align:left; line-height:120%">
    <br> 
        <b>
        Name: Kirit Dhillon, Sagar Lal
        </b>
    <br> 
        <b>
        Uni: ksd2142, sl3946
        </b>
</p>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Data Loanding and Exploration

In [3]:
data = pd.read_csv("winemag-data-130k-v2.csv")

# Remove non-usefule columns
data = data.drop(['taster_name', 'taster_twitter_handle'], axis=1)

In [4]:
print(data.dtypes)
print(data.shape)

Unnamed: 0       int64
country         object
description     object
designation     object
points           int64
price          float64
province        object
region_1        object
region_2        object
title           object
variety         object
winery          object
dtype: object
(129971, 12)


In [5]:
from sklearn.model_selection import train_test_split

#Split before deciding between text and non-text data to allow for joining later
X_trainval, X_test, y_trainval, y_test = train_test_split(data.drop(['points'], axis=1), data['points'])

In [6]:
text_X_trainval = X_trainval['description']
text_X_test = X_test['description']

non_text_X_trainval = X_trainval.drop(['description'], axis=1).reset_index().drop(['index', 'Unnamed: 0'], axis=1)
non_text_X_test = X_test.drop(['description'], axis=1).reset_index().drop(['index', 'Unnamed: 0'], axis=1)

In [7]:
print(text_X_trainval.shape, y_trainval.shape)
print(text_X_test.shape, y_test.shape)
print(non_text_X_trainval.shape)
print(non_text_X_test.shape)

(97478,) (97478,)
(32493,) (32493,)
(97478, 9)
(32493, 9)


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from gensim.models import Word2Vec

In [9]:
nt_continuous_features = ['price']
nt_categorical_features = ['country', 'title', 'designation', 'province', 'region_1', 'region_2',
                        'variety', 'winery']

In [10]:
base_categorical_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = 'constant', fill_value = "missing")),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

base_continuous_transformer = Pipeline(steps=[('simpleimputer', SimpleImputer(strategy = "median")),
                                         ('scaler', StandardScaler())])

base_continuous_and_categorical_preprocessor = ColumnTransformer(
    transformers=[
        ('num', base_continuous_transformer, nt_continuous_features),
        ('cat', base_categorical_transformer, nt_categorical_features)
    ])

## Word Vectors